In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import h5py
import time
from IPython import display

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [13]:
# Pattern set defining PDE parameters:
#   ['gliders', 'bubbles', 'maze', 'worms', 'spirals', 'spots']
pattern = 'bubbles' #'gliders'

# Initial condition trajectory:
#   000-099 = random Gaussians
#   100-199 = random Fourier series
traj = 82

# Size of upsampled grid (e.g. for plotting)
nup = 512

# Plot cadence
cadence = 5

if pattern == 'gliders':
    F, k = 0.014, 0.054
elif pattern == 'bubbles':
    F, k = 0.098, 0.057
elif pattern == 'maze':
    F, k = 0.029, 0.057
elif pattern == 'worms':
    F, k = 0.058, 0.065
elif pattern == 'spirals':
    F, k = 0.018, 0.051
elif pattern == 'spots':
    F, k = 0.03, 0.062
else:
    raise ValueError('Unknown pattern')

n = 128
dt = 10
nsteps = 1001
h5file = "data/{}_F={}_k={}.h5".format(pattern, F, k)
with h5py.File(h5file, mode='r') as hf:
    u = hf['u'][:]#[traj]
    v = hf['v'][:]#[traj]

# with h5py.File(h5file, mode='r') as hf:
#     x = np.linspace(-1, 1, nup, endpoint=False)
#     y = np.linspace(-1, 1, nup, endpoint=False)
#     for it in range(0, nsteps, cadence):
#         fig, (ax1, ax2) = plt.subplots(1, 2)
#         u = hf['u'][traj,it,:,:]
#         v = hf['v'][traj,it,:,:]
#         u = sp.signal.resample(u, nup, axis=0)
#         u = sp.signal.resample(u, nup, axis=1)
#         v = sp.signal.resample(v, nup, axis=0)
#         v = sp.signal.resample(v, nup, axis=1)
#         plot1 = ax1.pcolormesh(x, y, u)
#         plot2 = ax2.pcolormesh(x, y, v)
#         ax1.set(title='u', xlabel='x', ylabel='y',  xticks=[-1,0,1], yticks=[-1,0,1], aspect='equal')
#         ax2.set(title='v', xlabel='x', ylabel=None, xticks=[-1,0,1], yticks=[],       aspect='equal')
#         fig.suptitle('t = {}s'.format(it*dt), y=0.9)
#         fig.colorbar(plot1, orientation='horizontal')
#         fig.colorbar(plot2, orientation='horizontal')
#         plt.subplots_adjust(wspace=0.2)
#         display.display(plt.gcf())
#         time.sleep(0.05)
#         plt.close()
#         display.clear_output(wait=True)

In [15]:
u.shape

(200, 1001, 128, 128)

In [22]:
import os
import imageio
import numpy as np
import matplotlib.pyplot as plt
import shutil
from tqdm import tqdm
def create_gif(time_series, saving_directory, name_file = 'concentration', delete_imgs=False):
    time_series_min = time_series.min()
    time_series_max = time_series.max()
    if time_series.ndim > 3:
        #error:
        print('Error: The time series should be (time, height, width)')
        return
    if not os.path.exists(saving_directory + '/img_for_gif'):
        os.makedirs(saving_directory + '/img_for_gif')
    images = []
    for i in range(int(time_series.shape[0]/(10))):
        plt.imshow(time_series[i*10], origin='lower', vmin=time_series_min, vmax=time_series_max);
        plt.axis('off')
        plt.savefig(saving_directory + f'/img_for_gif/time_series_{i}.png', bbox_inches='tight', pad_inches=0)
        plt.close()
        images.append(imageio.imread(saving_directory + f'/img_for_gif/time_series_{i}.png'))

    imageio.mimsave(saving_directory + '/' + name_file + '.gif', images, duration=0.1)
    if delete_imgs:
        shutil.rmtree(saving_directory + '/img_for_gif')

In [23]:
create_gif(u[0], 'gif_data', name_file='u_gaussians', delete_imgs=True)
create_gif(v[0], 'gif_data', name_file='v_gaussians', delete_imgs=True)
create_gif(u[102], 'gif_data', name_file='u_fourier', delete_imgs=True)
create_gif(v[102], 'gif_data', name_file='v_fourier', delete_imgs=True)

/tmp/ipykernel_3891716/1755128613.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(saving_directory + f'/img_for_gif/time_series_{i}.png'))
